In [3]:
import os
import subprocess
import json
from pathlib import Path

In [2]:
def get_lossmap_paths(path, file_name_base):

    n_jobs = int(subprocess.check_output(f'find {path}/job* -maxdepth 0 -type d | wc -l', shell=True))
    n_jobs_succeeded = int(subprocess.check_output(f'find {path}/job* -maxdepth 0 -type d -not -empty | wc -l', shell=True))
    
    job_files_list = []
    for i in range(n_jobs):
        current_file_path = (subprocess.check_output(f'echo {path}/job_{i}/{file_name_base}.json', shell=True)).decode('ascii').strip()
        #current_file_path = (subprocess.check_output(f'echo {path}/job_{i}/{file_name_base}_{plane}.json', shell=True)).decode('ascii').strip()
        
        if os.path.exists(current_file_path):
            job_files_list.append(current_file_path)
        else:
            print(f'Job {i} failed')
    
    print(f'{n_jobs_succeeded} out of a total of {n_jobs} succeeded')
    if n_jobs_succeeded != len(job_files_list):
        print('ERROR: not all succeeded jobs paths were retrieved')
        
    return job_files_list

In [4]:
dir_path = Path('/eos/user/l/lpauwels/ht_condor_sps_tracking_results/tidvg_studies/aperture_measurements')

In [6]:
subdirs = {
    'delta_gap0.0': Path(dir_path, 'delta_gap_0.0'),
    'delta_gap0.1': Path(dir_path, 'delta_gap_0.1'),
    'delta_gap0.2': Path(dir_path, 'delta_gap_0.2'),
    'delta_gap0.3': Path(dir_path, 'delta_gap_0.3'),
    'delta_gap0.4': Path(dir_path, 'delta_gap_0.4'),
    'delta_gap0.5': Path(dir_path, 'delta_gap_0.5'),
    'delta_gap0.6': Path(dir_path, 'delta_gap_0.6'),
    'delta_gap0.7': Path(dir_path, 'delta_gap_0.7'),
    'delta_gap0.8': Path(dir_path, 'delta_gap_0.8'),
    'delta_gap0.9': Path(dir_path, 'delta_gap_0.9'),
    'delta_gap1.0': Path(dir_path, 'delta_gap_1.0'),
    'delta_gap1.1': Path(dir_path, 'delta_gap_1.1'),
    'delta_gap1.2': Path(dir_path, 'delta_gap_1.2'),
    'delta_gap1.3': Path(dir_path, 'delta_gap_1.3'),
    'delta_gap1.4': Path(dir_path, 'delta_gap_1.4'),
    'delta_gap1.5': Path(dir_path, 'delta_gap_1.5'),
    'delta_gap1.6': Path(dir_path, 'delta_gap_1.6'),
    'delta_gap1.7': Path(dir_path, 'delta_gap_1.7'),
    'delta_gap1.8': Path(dir_path, 'delta_gap_1.8'),
    'delta_gap1.9': Path(dir_path, 'delta_gap_1.9'),
    'delta_gap2.0': Path(dir_path, 'delta_gap_2.0'),
    'delta_gap2.1': Path(dir_path, 'delta_gap_2.1'),
    'delta_gap2.2': Path(dir_path, 'delta_gap_2.2'),
    'delta_gap2.3': Path(dir_path, 'delta_gap_2.3'),
}

In [8]:
lm_paths = {}
for delta_gap in subdirs:
    lm_paths[delta_gap] = {}
    lm_paths[delta_gap]['linear'] = get_lossmap_paths(subdirs[delta_gap], f'LM_linear_{delta_gap}')
    lm_paths[delta_gap]['ripple'] = get_lossmap_paths(subdirs[delta_gap], f'LM_ripple_{delta_gap}')

80 out of a total of 80 succeeded
80 out of a total of 80 succeeded
Job 21 failed
Job 28 failed
78 out of a total of 80 succeeded
Job 21 failed
Job 28 failed
78 out of a total of 80 succeeded
80 out of a total of 80 succeeded
80 out of a total of 80 succeeded
80 out of a total of 80 succeeded
80 out of a total of 80 succeeded
80 out of a total of 80 succeeded
80 out of a total of 80 succeeded
80 out of a total of 80 succeeded
80 out of a total of 80 succeeded
80 out of a total of 80 succeeded
80 out of a total of 80 succeeded
80 out of a total of 80 succeeded
80 out of a total of 80 succeeded
80 out of a total of 80 succeeded
80 out of a total of 80 succeeded
80 out of a total of 80 succeeded
80 out of a total of 80 succeeded
80 out of a total of 80 succeeded
80 out of a total of 80 succeeded
80 out of a total of 80 succeeded
80 out of a total of 80 succeeded
80 out of a total of 80 succeeded
80 out of a total of 80 succeeded
80 out of a total of 80 succeeded
80 out of a total of 80 su

In [11]:
combined_data = {}

for delta_gap in subdirs:
    for sim_type in ['ripple', 'linear']:
        print(f'{delta_gap}_{sim_type}')
        
        loc_dic = {
            'collimator': {'s': [], 'name': [], 'length': [], 'n': []},
            'aperture': {'s': [], 'name': [], 'n': []},
            'machine_length': 6911.5038,
            'interpolation': 0.1,
            'reversed': False
        }

        collimator_dict = {}  # To sum losses correctly
        aperture_dict = {}    # To sum losses correctly

        for file_path in lm_paths[delta_gap][sim_type]:
            with open(file_path, "r") as f:
                data = json.load(f)

            # --- Merge collimator data ---
            for i in range(len(data['collimator']['s'])):
                key = (data['collimator']['s'][i], data['collimator']['name'][i])
                if key in collimator_dict:
                    collimator_dict[key]["n"] += data['collimator']['n'][i]
                else:
                    collimator_dict[key] = {
                        "s": data['collimator']['s'][i],
                        "name": data['collimator']['name'][i],
                        "length": data['collimator']['length'][i],
                        "n": data['collimator']['n'][i]
                    }

            # --- Merge aperture data ---
            for i in range(len(data['aperture']['s'])):
                key = (data['aperture']['s'][i], data['aperture']['name'][i])
                if key in aperture_dict:
                    aperture_dict[key]["n"] += data['aperture']['n'][i]
                else:
                    aperture_dict[key] = {
                        "s": data['aperture']['s'][i],
                        "name": data['aperture']['name'][i],
                        "n": data['aperture']['n'][i]
                    }

        # Convert collimator dictionary back to lists
        sorted_collimator = sorted(collimator_dict.values(), key=lambda x: x["s"])
        loc_dic['collimator']['s'] = [entry["s"] for entry in sorted_collimator]
        loc_dic['collimator']['name'] = [entry["name"] for entry in sorted_collimator]
        loc_dic['collimator']['length'] = [entry["length"] for entry in sorted_collimator]
        loc_dic['collimator']['n'] = [entry["n"] for entry in sorted_collimator]

        # Convert aperture dictionary back to lists
        sorted_aperture = sorted(aperture_dict.values(), key=lambda x: x["s"])
        loc_dic['aperture']['s'] = [entry["s"] for entry in sorted_aperture]
        loc_dic['aperture']['name'] = [entry["name"] for entry in sorted_aperture]
        loc_dic['aperture']['n'] = [entry["n"] for entry in sorted_aperture]

        combined_data[f'{delta_gap}_{sim_type}'] = loc_dic
        with open(f'tidvg_studies/aperture_measurements/combined_jsons/{delta_gap}_{sim_type}.json', 'w') as f:
            json.dump(loc_dic, f, indent=4)


delta_gap0.0_ripple
delta_gap0.0_linear
delta_gap0.1_ripple
delta_gap0.1_linear
delta_gap0.2_ripple
delta_gap0.2_linear
delta_gap0.3_ripple
delta_gap0.3_linear
delta_gap0.4_ripple
delta_gap0.4_linear
delta_gap0.5_ripple
delta_gap0.5_linear
delta_gap0.6_ripple
delta_gap0.6_linear
delta_gap0.7_ripple
delta_gap0.7_linear
delta_gap0.8_ripple
delta_gap0.8_linear
delta_gap0.9_ripple
delta_gap0.9_linear
delta_gap1.0_ripple
delta_gap1.0_linear
delta_gap1.1_ripple
delta_gap1.1_linear
delta_gap1.2_ripple
delta_gap1.2_linear
delta_gap1.3_ripple
delta_gap1.3_linear
delta_gap1.4_ripple
delta_gap1.4_linear
delta_gap1.5_ripple
delta_gap1.5_linear
delta_gap1.6_ripple
delta_gap1.6_linear
delta_gap1.7_ripple
delta_gap1.7_linear
delta_gap1.8_ripple
delta_gap1.8_linear
delta_gap1.9_ripple
delta_gap1.9_linear
delta_gap2.0_ripple
delta_gap2.0_linear
delta_gap2.1_ripple
delta_gap2.1_linear
delta_gap2.2_ripple
delta_gap2.2_linear
delta_gap2.3_ripple
delta_gap2.3_linear
